MobileNetV3

In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
from sklearn.utils import class_weight

# --- 1. Configuration for this Experiment ---
MODEL_CHOICE = "MobileNetV3Large"
INPUT_SIZE = 224
BATCH_SIZE = 8  # Best performing batch size for MobileNet family
INITIAL_LR = 1e-4  # A stable learning rate for the initial stage
FINE_TUNE_LR = 1e-5 # A very low learning rate for the fine-tuning stage
FEATURE_EXTRACTION_EPOCHS = 50
FINE_TUNE_EPOCHS = 50
PATIENCE = 15 # Increased patience for more stable training
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_ultimate_finetune_bs{BATCH_SIZE}'
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting Ultimate Experiment for: {MODEL_NAME} ---")
print("Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning")
print("="*80)

# --- 2. Data Loading ---
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model ---
base_model = MobileNetV3Large(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# --- STAGE 1: STABLE FEATURE EXTRACTION ---
print("\n--- STAGE 1: STABLE FEATURE EXTRACTION ---")
base_model.trainable = False

inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training the classification head...")
history = model.fit(
    X_train, y_train,
    epochs=FEATURE_EXTRACTION_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)]
)

# --- STAGE 2: ADVANCED FINE-TUNING ---
print("\n--- STAGE 2: ADVANCED FINE-TUNING ---")

# --- Calculate Class Weights (introduced only in Stage 2) ---
print("\nCalculating class weights for fine-tuning...")
class_labels = np.unique(y_train)
weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
class_weights_dict = dict(zip(class_labels, weights))
print("Class Weights:", class_weights_dict)

# --- Unfreeze layers for fine-tuning ---
base_model.trainable = True
fine_tune_at = math.ceil(len(base_model.layers) * 0.5) # Unfreeze top 50%
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing from layer {fine_tune_at} onwards.")

# --- Re-compile with a very low learning rate ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for the fine-tuning stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE + 5, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("Fine-tuning the model with class weights and a low learning rate...")
total_epochs = history.epoch[-1] + 1 + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict, # Introduce class weights here
    callbacks=callbacks_finetune
)

# Load the best model saved during fine-tuning
model = tf.keras.models.load_model(checkpoint_path)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")


--- Starting Ultimate Experiment for: MobileNetV3Large_ultimate_finetune_bs8 ---
Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning

Loading preprocessed data from 'processed_data\BrinjalFruitX'...
Data loaded successfully.

--- STAGE 1: STABLE FEATURE EXTRACTION ---
Training the classification head...
Epoch 1/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 17s 78ms/step - accuracy: 0.2751 - loss: 1.6719 - val_accuracy: 0.4033 - val_loss: 1.4530
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.3632 - loss: 1.5562 - val_accuracy: 0.4033 - val_loss: 1.4331
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.3516 - loss: 1.5756 - val_accuracy: 0.4033 - val_loss: 1.4276
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.3394 - loss: 1.5812 - val_accuracy: 0.4033 - val_loss: 1.4267
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.3634 - loss: 1.5103 - val_accuracy: 0.4033 - val_loss: 1.4265
Epoch 6/50
158/158 ━━

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war


Updating summary results file...

--- Experiment for MobileNetV3Large_ultimate_finetune_bs8 is complete. Best model saved to results\MobileNetV3Large_ultimate_finetune_bs8\best_model.keras ---


In [2]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
from sklearn.utils import class_weight

# --- 1. Configuration for this Experiment ---
MODEL_CHOICE = "MobileNetV3Large"
INPUT_SIZE = 224
BATCH_SIZE = 8  # Best performing batch size for MobileNet family
INITIAL_LR = 1e-4  # A stable learning rate for the initial stage
FINE_TUNE_LR = 1e-5 # A very low learning rate for the fine-tuning stage
FEATURE_EXTRACTION_EPOCHS = 50
FINE_TUNE_EPOCHS = 50
PATIENCE = 15 # Increased patience for more stable training
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_ultimate_finetune_bs{BATCH_SIZE}'
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting Ultimate Experiment for: {MODEL_NAME} ---")
print("Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning")
print("="*80)

# --- 2. Data Loading ---
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model ---
base_model = MobileNetV3Large(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# --- STAGE 1: STABLE FEATURE EXTRACTION ---
print("\n--- STAGE 1: STABLE FEATURE EXTRACTION ---")
base_model.trainable = False

inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training the classification head...")
history = model.fit(
    X_train, y_train,
    epochs=FEATURE_EXTRACTION_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)]
)

# --- STAGE 2: ADVANCED FINE-TUNING ---
print("\n--- STAGE 2: ADVANCED FINE-TUNING ---")

# --- Calculate Class Weights (introduced only in Stage 2) ---
print("\nCalculating class weights for fine-tuning...")
class_labels = np.unique(y_train)
weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
class_weights_dict = dict(zip(class_labels, weights))
print("Class Weights:", class_weights_dict)

# --- Unfreeze layers for fine-tuning ---
base_model.trainable = True
fine_tune_at = math.ceil(len(base_model.layers) * 0.5) # Unfreeze top 50%
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing from layer {fine_tune_at} onwards.")

# --- Re-compile with a very low learning rate ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for the fine-tuning stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE + 5, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("Fine-tuning the model with class weights and a low learning rate...")
total_epochs = history.epoch[-1] + 1 + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict, # Introduce class weights here
    callbacks=callbacks_finetune
)

# Load the best model saved during fine-tuning
model = tf.keras.models.load_model(checkpoint_path)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")


--- Starting Ultimate Experiment for: MobileNetV3Large_ultimate_finetune_bs8 ---
Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning

Loading preprocessed data from 'processed_data\BrinjalFruitX_balanced'...
Data loaded successfully.

--- STAGE 1: STABLE FEATURE EXTRACTION ---
Training the classification head...
Epoch 1/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 27s 71ms/step - accuracy: 0.2090 - loss: 1.7872 - val_accuracy: 0.2762 - val_loss: 1.6015
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 22s 68ms/step - accuracy: 0.2158 - loss: 1.7212 - val_accuracy: 0.4586 - val_loss: 1.5910
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.2153 - loss: 1.7093 - val_accuracy: 0.3978 - val_loss: 1.6012
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 21s 67ms/step - accuracy: 0.2075 - loss: 1.6786 - val_accuracy: 0.4696 - val_loss: 1.5897
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.2044 - loss: 1.6599 - val_accuracy: 0.4917 - val_loss: 1.5944
Epoch 6/50
3

In [3]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
from sklearn.utils import class_weight

# --- 1. Configuration for this Experiment ---
MODEL_CHOICE = "MobileNetV3Large"
INPUT_SIZE = 224
BATCH_SIZE = 8  # Best performing batch size for MobileNet family
INITIAL_LR = 1e-4  # A stable learning rate for the initial stage
FINE_TUNE_LR = 1e-5 # A very low learning rate for the fine-tuning stage
FEATURE_EXTRACTION_EPOCHS = 50
FINE_TUNE_EPOCHS = 50
PATIENCE = 15 # Increased patience for more stable training
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_ultimate_finetune_bs{BATCH_SIZE}_3'
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting Ultimate Experiment for: {MODEL_NAME} ---")
print("Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning")
print("="*80)

# --- 2. Data Loading ---
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced_classless')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model ---
base_model = MobileNetV3Large(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# --- STAGE 1: STABLE FEATURE EXTRACTION ---
print("\n--- STAGE 1: STABLE FEATURE EXTRACTION ---")
base_model.trainable = False

inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training the classification head...")
history = model.fit(
    X_train, y_train,
    epochs=FEATURE_EXTRACTION_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)]
)

# --- STAGE 2: ADVANCED FINE-TUNING ---
print("\n--- STAGE 2: ADVANCED FINE-TUNING ---")

# --- Calculate Class Weights (introduced only in Stage 2) ---
print("\nCalculating class weights for fine-tuning...")
class_labels = np.unique(y_train)
weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
class_weights_dict = dict(zip(class_labels, weights))
print("Class Weights:", class_weights_dict)

# --- Unfreeze layers for fine-tuning ---
base_model.trainable = True
fine_tune_at = math.ceil(len(base_model.layers) * 0.5) # Unfreeze top 50%
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing from layer {fine_tune_at} onwards.")

# --- Re-compile with a very low learning rate ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for the fine-tuning stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE + 5, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("Fine-tuning the model with class weights and a low learning rate...")
total_epochs = history.epoch[-1] + 1 + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict, # Introduce class weights here
    callbacks=callbacks_finetune
)

# Load the best model saved during fine-tuning
model = tf.keras.models.load_model(checkpoint_path)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")


--- Starting Ultimate Experiment for: MobileNetV3Large_ultimate_finetune_bs8_3 ---
Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning

Loading preprocessed data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- STAGE 1: STABLE FEATURE EXTRACTION ---
Training the classification head...
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 77ms/step - accuracy: 0.4394 - loss: 0.8671 - val_accuracy: 0.5887 - val_loss: 0.6910
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.4695 - loss: 0.7680 - val_accuracy: 0.3710 - val_loss: 0.7000
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.5018 - loss: 0.7524 - val_accuracy: 0.4113 - val_loss: 0.7120
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.5359 - loss: 0.7214 - val_accuracy: 0.4113 - val_loss: 0.7017
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.4890 - loss: 0.7438 - val_accuracy: 0.4597 - val_loss: 0.6939
Epoc

In [4]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
# --- MODIFIED ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
from sklearn.utils import class_weight

# --- 1. Configuration for this Experiment ---
# --- MODIFIED ---
MODEL_CHOICE = "MobileNetV2"
INPUT_SIZE = 224
BATCH_SIZE = 8  # Best performing batch size for MobileNet family
INITIAL_LR = 1e-4  # A stable learning rate for the initial stage
FINE_TUNE_LR = 1e-5 # A very low learning rate for the fine-tuning stage
FEATURE_EXTRACTION_EPOCHS = 50
FINE_TUNE_EPOCHS = 50
PATIENCE = 15 # Increased patience for more stable training
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_ultimate_finetune_bs{BATCH_SIZE}' # --- MODIFIED ---
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting Ultimate Experiment for: {MODEL_NAME} ---")
print("Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning")
print("="*80)

# --- 2. Data Loading ---
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced_classless')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model ---
# --- MODIFIED ---
base_model = MobileNetV2(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# --- STAGE 1: STABLE FEATURE EXTRACTION ---
print("\n--- STAGE 1: STABLE FEATURE EXTRACTION ---")
base_model.trainable = False

inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training the classification head...")
history = model.fit(
    X_train, y_train,
    epochs=FEATURE_EXTRACTION_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)]
)

# --- STAGE 2: ADVANCED FINE-TUNING ---
print("\n--- STAGE 2: ADVANCED FINE-TUNING ---")

# --- Calculate Class Weights (introduced only in Stage 2) ---
print("\nCalculating class weights for fine-tuning...")
class_labels = np.unique(y_train)
weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
class_weights_dict = dict(zip(class_labels, weights))
print("Class Weights:", class_weights_dict)

# --- Unfreeze layers for fine-tuning ---
base_model.trainable = True
fine_tune_at = math.ceil(len(base_model.layers) * 0.5) # Unfreeze top 50%
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing from layer {fine_tune_at} onwards.")

# --- Re-compile with a very low learning rate ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for the fine-tuning stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE + 5, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("Fine-tuning the model with class weights and a low learning rate...")
total_epochs = history.epoch[-1] + 1 + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict, # Introduce class weights here
    callbacks=callbacks_finetune
)

# Load the best model saved during fine-tuning
model = tf.keras.models.load_model(checkpoint_path)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")



--- Starting Ultimate Experiment for: MobileNetV2_ultimate_finetune_bs8 ---
Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning

Loading preprocessed data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- STAGE 1: STABLE FEATURE EXTRACTION ---
Training the classification head...
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 85ms/step - accuracy: 0.5422 - loss: 0.8895 - val_accuracy: 0.6532 - val_loss: 0.6075
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - accuracy: 0.6274 - loss: 0.7075 - val_accuracy: 0.6935 - val_loss: 0.5417
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.7126 - loss: 0.5577 - val_accuracy: 0.7258 - val_loss: 0.5027
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 10s 77ms/step - accuracy: 0.7501 - loss: 0.5146 - val_accuracy: 0.7339 - val_loss: 0.4648
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.8057 - loss: 0.4536 - val_accuracy: 0.7258 - val_loss: 0.5047
Epoch 6/

In [5]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
# --- MODIFIED ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
from sklearn.utils import class_weight

# --- 1. Configuration for this Experiment ---
# --- MODIFIED ---
MODEL_CHOICE = "MobileNetV2"
INPUT_SIZE = 224
BATCH_SIZE = 8  # Best performing batch size for MobileNet family
INITIAL_LR = 1e-4  # A stable learning rate for the initial stage
FINE_TUNE_LR = 1e-5 # A very low learning rate for the fine-tuning stage
FEATURE_EXTRACTION_EPOCHS = 50
FINE_TUNE_EPOCHS = 50
PATIENCE = 15 # Increased patience for more stable training
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_ultimate_finetune_bs{BATCH_SIZE}' # --- MODIFIED ---
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting Ultimate Experiment for: {MODEL_NAME} ---")
print("Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning")
print("="*80)

# --- 2. Data Loading ---
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model ---
# --- MODIFIED ---
base_model = MobileNetV2(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights='imagenet')

# --- STAGE 1: STABLE FEATURE EXTRACTION ---
print("\n--- STAGE 1: STABLE FEATURE EXTRACTION ---")
base_model.trainable = False

inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Training the classification head...")
history = model.fit(
    X_train, y_train,
    epochs=FEATURE_EXTRACTION_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=1)]
)

# --- STAGE 2: ADVANCED FINE-TUNING ---
print("\n--- STAGE 2: ADVANCED FINE-TUNING ---")

# --- Calculate Class Weights (introduced only in Stage 2) ---
print("\nCalculating class weights for fine-tuning...")
class_labels = np.unique(y_train)
weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
class_weights_dict = dict(zip(class_labels, weights))
print("Class Weights:", class_weights_dict)

# --- Unfreeze layers for fine-tuning ---
base_model.trainable = True
fine_tune_at = math.ceil(len(base_model.layers) * 0.5) # Unfreeze top 50%
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
print(f"Unfreezing from layer {fine_tune_at} onwards.")

# --- Re-compile with a very low learning rate ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for the fine-tuning stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks_finetune = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE + 5, verbose=1),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("Fine-tuning the model with class weights and a low learning rate...")
total_epochs = history.epoch[-1] + 1 + FINE_TUNE_EPOCHS
history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1] + 1,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict, # Introduce class weights here
    callbacks=callbacks_finetune
)

# Load the best model saved during fine-tuning
model = tf.keras.models.load_model(checkpoint_path)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")



--- Starting Ultimate Experiment for: MobileNetV2_ultimate_finetune_bs8 ---
Strategy: Hybrid Training with Class Weights, Augmentation, and Fine-Tuning

Loading preprocessed data from 'processed_data\BrinjalFruitX_balanced'...
Data loaded successfully.

--- STAGE 1: STABLE FEATURE EXTRACTION ---
Training the classification head...
Epoch 1/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 27s 74ms/step - accuracy: 0.2610 - loss: 1.9434 - val_accuracy: 0.5083 - val_loss: 1.2725
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.4265 - loss: 1.4831 - val_accuracy: 0.6740 - val_loss: 0.9897
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step - accuracy: 0.5292 - loss: 1.2174 - val_accuracy: 0.7403 - val_loss: 0.8657
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - accuracy: 0.5917 - loss: 1.0671 - val_accuracy: 0.7348 - val_loss: 0.7791
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 22s 69ms/step - accuracy: 0.6087 - loss: 0.9956 - val_accuracy: 0.7403 - val_loss: 0.7827
Epoch 6/50
317/31

C:\Users\dr-basab\AppData\Local\Temp\ipykernel_13812\1456964767.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8892' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
C:\Users\dr-basab\AppData\Local\Temp\ipykernel_13812\1456964767.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3726' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
C:\Users\dr-basab\AppData\Local\Temp\ipykernel_13812\1456964767.py:177: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8631' has dtype incompa

---
---
---
---

exp-2 ends here

In [8]:
# MobilNetV2 w/o transfer learning

import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
# --- MODIFIED ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math

# --- 1. Configuration for this Experiment ---
# --- MODIFIED ---
MODEL_CHOICE = "MobileNetV2"
INPUT_SIZE = 224
BATCH_SIZE = 8
LEARNING_RATE = 1e-4 # A single, stable learning rate for training from scratch
EPOCHS = 100
PATIENCE = 20 # Increased patience as training from scratch can take longer to converge
RESULTS_DIR = 'results'
MODEL_NAME = f'{MODEL_CHOICE}_from_scratch_bs{BATCH_SIZE}' # --- MODIFIED ---
model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
os.makedirs(model_results_dir, exist_ok=True)

print("="*80)
print(f"--- Starting 'From Scratch' Experiment for: {MODEL_NAME} ---")
print(f"Parameters: Batch Size={BATCH_SIZE}, Learning Rate={LEARNING_RATE}")
print("="*80)

# --- 2. Data Loading ---
# Using the balanced dataset you created
PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced_classless')
print(f"\nLoading preprocessed data from '{PROCESSED_DIR}'...")
try:
    X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
    y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
    X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
    y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
    X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
    y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
    with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
        class_names = json.load(f)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"ERROR: Data not found at {PROCESSED_DIR}.")
    exit()

# --- 3. Model Definition and Training ---

# --- Data Augmentation Layer ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
], name="data_augmentation")

# --- Base Model (Training from Scratch) ---
# --- MODIFIED ---
base_model = MobileNetV2(input_shape=(INPUT_SIZE, INPUT_SIZE, 3), include_top=False, weights=None) # No ImageNet weights
base_model.trainable = True # The entire model is trainable

# --- Full Model ---
inputs = layers.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True) # Set training=True as layers are trainable
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(len(class_names), activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks for a single training stage ---
checkpoint_path = os.path.join(model_results_dir, 'best_model.keras')
callbacks = [
    EarlyStopping(monitor='val_loss', patience=PATIENCE, verbose=1, restore_best_weights=True),
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)
]

print("\n--- Starting Single-Stage Training From Scratch ---")
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

# --- 4. Evaluation and Saving ---
print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
y_pred_test_probs = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
print("\nTest Set Classification Report:\n")
print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

# (The rest of the saving and summary code is the same)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv(os.path.join(model_results_dir, 'classification_report.csv'))

cm = confusion_matrix(y_test, y_pred_test_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title(f'{MODEL_NAME} Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
plt.close()

print("\nUpdating summary results file...")
summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
summary_data = {
    'model_name': MODEL_NAME,
    'batch_size': BATCH_SIZE,
    'test_accuracy': f"{test_accuracy:.4f}",
    'test_loss': f"{test_loss:.4f}",
    'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
    'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}"
}
new_results_df = pd.DataFrame([summary_data])
if os.path.exists(summary_file):
    summary_df = pd.read_csv(summary_file)
    if MODEL_NAME in summary_df['model_name'].values:
        summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
    else:
        summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
    summary_df.to_csv(summary_file, index=False)
else:
    new_results_df.to_csv(summary_file, index=False)

print(f"\n--- Experiment for {MODEL_NAME} is complete. Best model saved to {checkpoint_path} ---")

--- Starting 'From Scratch' Experiment for: MobileNetV2_from_scratch_bs8 ---
Parameters: Batch Size=8, Learning Rate=0.0001

Loading preprocessed data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- Starting Single-Stage Training From Scratch ---
Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.5806 - loss: 0.7581
Epoch 1: val_loss improved from inf to 0.68488, saving model to results\MobileNetV2_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 57s 287ms/step - accuracy: 0.5810 - loss: 0.7576 - val_accuracy: 0.5887 - val_loss: 0.6849 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.6721 - loss: 0.5967
Epoch 2: val_loss improved from 0.68488 to 0.67943, saving model to results\MobileNetV2_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 35s 274ms/step - accuracy: 0.6723 - loss: 0.5966 - val_accuracy: 0.5887 - val_loss: 0.6794 - learning_rate: 1.0000e-0

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war


--- Experiment for MobileNetV2_from_scratch_bs8 is complete. Best model saved to results\MobileNetV2_from_scratch_bs8\best_model.keras ---
